In [7]:
import os 
from collections import OrderedDict
import shutil

# Intialization
OUR_METHOD = 'feedback_F64T4B1U1G6'
save_dir = 'save_new'
#os.removedirs(save_dir) 

methods = OrderedDict()
record_mean = False

# Transfer to OrderedDict() for future use
with open('PSNR_SSIM_Results_BI_model.txt') as f:
    for line in f:
        if 'Method' in line:
            name = line.split()[-3][:-1]
            bm_name = line.split()[-1]
            if name not in methods.keys():
                methods[name] = OrderedDict()
            if bm_name not in methods[name].keys():
                methods[name][bm_name] = OrderedDict()
                methods[name][bm_name]['img_name'] = list()
                methods[name][bm_name]['PSNR'] = list()
                methods[name][bm_name]['SSIM'] = list()
                # TODO
                # methods[name][bm_name]['IFC'] = list()
        elif '--' in line: 
            record_mean = True
        elif '**' in line: continue
        elif record_mean:
            split_line = line.split()
            methods[name][bm_name]['avg_PSNR'] = float(split_line[-5])
            methods[name][bm_name]['avg_SSIM'] = float(split_line[-3])
            record_mean = False
        else:
            split_line = line.split()
            methods[name][bm_name]['img_name'].append(split_line[-7][:-1])
            methods[name][bm_name]['PSNR'].append(float(split_line[-5]))
            methods[name][bm_name]['SSIM'].append(float(split_line[-3]))

In [8]:
methods

OrderedDict([('Bicubic',
              OrderedDict([('Set5',
                            OrderedDict([('img_name',
                                          ['baby_Bicubic_x4.png',
                                           'bird_Bicubic_x4.png',
                                           'butterfly_Bicubic_x4.png',
                                           'head_Bicubic_x4.png',
                                           'woman_Bicubic_x4.png']),
                                         ('PSNR',
                                          [31.780735,
                                           30.184407,
                                           22.099731,
                                           31.60902,
                                           26.466839]),
                                         ('SSIM',
                                          [0.857112,
                                           0.873378,
                                           0.737098,
                 

In [9]:
bm_list = methods[OUR_METHOD].keys()
for bm_name in bm_list:
    save_bm_dir = os.path.join(save_dir, bm_name)
    if not os.path.exists(save_bm_dir):
        os.makedirs(save_bm_dir)
    our_results = methods[OUR_METHOD][bm_name]['PSNR']
    comp_total = [True for _ in range(len(our_results))]
   
    # Comparision (You can edit the comparision criterion here)
    for name in methods.keys():
        if name == OUR_METHOD: continue
        comp_method = list(map(lambda x,y:x > y, our_results, methods[name][bm_name]['PSNR']))
        comp_total = list(map(lambda x,y:x and y, comp_total, comp_method))
    comp_idx = [idx for idx, item in enumerate(comp_total) if item]
    
    # save files
    for name in methods.keys():
        if name == 'OurA+': continue
        for sel_idx in comp_idx:
            psnr = methods[name][bm_name]['PSNR'][sel_idx]
            old_img_name = methods[name][bm_name]['img_name'][sel_idx]
            new_img_name = os.path.splitext(old_img_name)[0] + '_%f'%psnr + os.path.splitext(old_img_name)[-1]
            src_path = os.path.join('SR', 'BI', name, bm_name, 'x4', old_img_name)
            shutil.copy(src_path, os.path.join(save_bm_dir, new_img_name))